In [3]:
# MASTER ONLY
from prog_edu_assistant_tools.summary_test_result import SummaryTestResult
# imports %%solution, %%submission, %%template
%load_ext prog_edu_assistant_tools.magics
from prog_edu_assistant_tools.magics import report, autotest

# Introduction to data frames

* What is a data frame?
    * Mostly a table, just like spreadsheets
   
* Example: loading data frame from CSV
    * Aside: what is CSV format
* A good dataframe
  * One variable = one column (i.e. no multiple temperature columns)
  * One observation = one row
  * Trade-offs of different definitions of "one observation"
* Exercise: create a good data frame from a textual description of the data
* Explore data frame
  * head/tail
  * describe
* Plotting with Plotly Express
* Exercise: plot something from the dataframe from the previous exercise
* Manipulating data
  * Filtering rows
  * Dropping columns
  * Adding new derived columns
* Split-apply-combine
* Exercise: clean up the data frame and apply a data transformation
* Main exercise
  * Given a data set in CSV, load it into a dataframe
  * Apply cleaning
  * Draw a few plots
  * Answer a qualitative question about the data based on the plots


In [4]:
import io

import numpy as np
import pandas as pd
import plotly_express as px

## Example: create the data frame from lists

You can create a data frame by using `DataFrame` constructor, and give it a dictionary with the data.
Each element in the data dictionary corresponds to a column. The dictionary key becomes the column name
and the dictionary value becomes the contents of the column. If the column has constant value,
it is possible to give the value directly instead of repeating it for every row.

In [3]:
df = pd.DataFrame({
    'weekday': ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'],
    'work': [True, True, True, True, False],
    'time': '9:00',
})
df

,weekday,work,time
0,Monday,True,9:00
1,Tuesday,True,9:00
2,Wednesday,True,9:00
3,Thursday,True,9:00
4,Friday,False,9:00


## Example: load the data frame from CSV file (inline)

In the below example, we use CSV format and prepare the data as multi-line string literal.
Important trivia:
* `"""` is used to define multi-line string literals. Newlines embedded in the literal
  are parts of the value.
* `io.StringIO` is used to create a file-like object that returns the string literal data
  on a read.
* `pd.read_csv` takes a file name or a file-like object to read the data in CSV format.

In [4]:
CSV = """weekday,arrive,depart,work
Monday,9:00,17:00,true
Tuesday,8:45,16:20,true
Wednesday,8:53,17:11,true
Thursday,10:00,19:50,true
Friday,9:00,16:00,true
"""

df = pd.read_csv(io.StringIO(CSV))
df

,weekday,arrive,depart,work
0,Monday,9:00,17:00,True
1,Tuesday,8:45,16:20,True
2,Wednesday,8:53,17:11,True
3,Thursday,10:00,19:50,True
4,Friday,9:00,16:00,True


## Exercise: Create a data frame

**TODO(salikh): Civil time is not easy to represent and handle, maybe rework this example to use plain numbers.**

In the next cell, create a data frame representing the data described below. You can create the data frame either by constructing it from lists or by parsing an inline CSV data, at your option.

Alice tries to analyze her habits about the time she comes to work and leaves.

* Alice comes to office at 9:00 every weekday, except Wednesdays, when she is late by 30 minutes.
* Alice does not do work on Fridays, but still comes to office to water a plant and chat, which takes 60 minutes.
* Alice goes to gym every Tuesday and Thursday
* Alice leaves the office at 17:00 when she goes to the gym, and at 18:00 otherwise.

Please define a data frame with the following columns:
* `weekday` --- The name of the weekday (Monday, Tuesday, etc.)
* `arrive` -- The time of coming to the office
* `depart` -- The time of leaving the office
* `work` -- The boolean indicating whether Alice works
* `gym` -- The boolean indicating whether Alice goes to gym

In [5]:
%%solution
CSV = """day,weekday,arrive,depart,work,gym
1,Monday,9:00,18:00,true,false
2,Tuesday,9:00,17:00,true,true
3,Wednesday,9:30,18:00,true,false
4,Thursday,9:00,17:00,true,true
5,Friday,9:00,10:00,false,false
"""

df = pd.read_csv(io.StringIO(CSV))

In [6]:
# Show the data frame.
df

,day,weekday,arrive,depart,work,gym
0,1,Monday,9:00,18:00,True,False
1,2,Tuesday,9:00,17:00,True,True
2,3,Wednesday,9:30,18:00,True,False
3,4,Thursday,9:00,17:00,True,True
4,5,Friday,9:00,10:00,False,False


## Transforming the data frame

A typical scenario for transforming data in a data frame is conversion of the data
into a format that is easier to handle when computing or plotting.

A very granular example of transformation is using 

In [10]:
def time_to_minutes(t):
    """Converts timestamp given as string in "HH:MM" format into the number of minutes since midnight.
    
    Returns None if the time cannot be parsed.
    """
    parts = t.split(':')
    if len(parts) != 2: return None
    nums = [int(x) for x in parts]
    return np.sum(np.array(nums) * [60, 1])

df['arrive_min'] = df['arrive'].apply(time_to_minutes) 
df['depart_min'] = df['depart'].apply(time_to_minutes)
df

,day,weekday,arrive,depart,work,gym,arrive_min,depart_min,in_office_min
0,1,Monday,9:00,18:00,True,False,540,1080,540
1,2,Tuesday,9:00,17:00,True,True,540,1020,480
2,3,Wednesday,9:30,18:00,True,False,570,1080,510
3,4,Thursday,9:00,17:00,True,True,540,1020,480
4,5,Friday,9:00,10:00,False,False,540,600,60


The new columns can also be created using element-wise arithmetic operations on existing columns.

In [ ]:
df['in_office_min'] = df['depart_min'] - df['arrive_min']
df

In [13]:
px.bar(df, x='weekday', y='in_office_min')

In [17]:
 px.line(df, x='weekday', y='arrive_min')

In [19]:
df1 = pd.DataFrame(data={'weekday': df['weekday'], 'time_min': df['arrive_min'], 'what': 'arrive'})
df2 = pd.DataFrame(data={'weekday': df['weekday'], 'time_min': df['depart_min'], 'what': 'depart'})
df3 = pd.concat([df1, df2])
df3
df3

,weekday,time_min,what
0,Monday,540,arrive
1,Tuesday,540,arrive
2,Wednesday,570,arrive
3,Thursday,540,arrive
4,Friday,540,arrive
0,Monday,1080,depart
1,Tuesday,1020,depart
2,Wednesday,1080,depart
3,Thursday,1020,depart
4,Friday,600,depart


In [29]:
px.line(df3, x='weekday', y='time_min', color='what')

# Random tidbits

### CSV in local file via `%%writefile`

There is a way to work with CSV files in a traditional way, but still be able to define them directly in the Jupyter interface with `%%writefile` magic. We do not recommend it because it makes harder our task of automatic assessment of correctness of submitted notebooks.

In [5]:
%%writefile test.csv
x,y
1,2
3,4

Overwriting test.csv


In [6]:
test_df = pd.read_csv('test.csv')
test_df

,x,y
0,1,2
1,3,4
